In [1]:
%config IPCompleter.greedy=True

In [60]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import csv

data_path = "./weapon_data.csv"

In [63]:
def getCsvHeader(filename):
    with open(filename, mode='r') as f:
        reader = csv.reader(f)
        header = next(reader)
        return header
    
def getCsvAsDict(filename):
    header = getCsvHeader(filename)
    result = { h: [] for h in header }
    with open(filename, mode='r') as file:
        for row in csv.DictReader(file):
            for h in header:
                result[h].append(row[h])

    return result

In [64]:
features = getCsvAsDict(data_path)
print(len(features['weapshowname']))
#print(features)

148


In [24]:
def tensorToParameterDict(tensor, cols_to_vars_dict):
    idx =  0 #keep track of the id to find the right value in the processed tensor
    result = dict()
    for key in cols_to_vars_dict:
        '''
        e.g., key = _NumericColumn(key='bdrop', shape=(1,), default_value=None, 
                    dtype=tf.float32, normalizer_fn=None)
        then key[0] = 'bdrop'
        '''
        if key[0] in NUMERICAL_PARAMS:
            result[key[0]] = str(tensor[idx])
        
            '''
            e.g., key = _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
            then key[0] = _VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
            and key[0][0] = 'firemode'
            and key[0][0][0] = 'Automatic'
            '''
        elif (len(key[0]) > 0) and key[0][0] in CATEGORICAL_PARAMS:
            for i in range(0, len(key[0][1])-1):
                param = key[0][0] + "_" + key[0][1][i]
                result[param] = str(tensor[idx])
                idx += 1
                
        idx += 1
    return result

In [37]:
#define constants for data processing
CATEGORICAL_PARAMS  = ['type', 'firemode', 'ammo']
NUMERICAL_PARAMS = ['rof', 'damages_first', 'damages_last', 'dmg_distances_first', 'dmg_distances_last', 'initialspeed',
                    'drag', 'magsize', 'shotspershell', 'reloadempty', 'hordispersion', 'verdispersion', 
                    'hiprecoildec', 'hiprecoilright', 'hiprecoilup' ]
WEAPON_TYPES = ['Shotgun', 'Pistol', 'Rifle', 'Submachine Gun', 'Sniper Rifle', 'Light Machine Gun']
WEAPON_FIREMODES = ['Automatic', 'Semi-Automatic', 'Single-Action']

#convert all numerical data to float so they can be used as 'tf.feature_column.numeric_column'
for key, values in features.items():
    if key in NUMERICAL_PARAMS:
        features[key] = [float('{:.4f}'.format(float(value))) for value in values]

#encode categorical data
type_column = tf.feature_column.categorical_column_with_vocabulary_list('type', WEAPON_TYPES)
type_column = tf.feature_column.indicator_column(type_column)

firemode_colum = tf.feature_column.categorical_column_with_vocabulary_list('firemode', WEAPON_FIREMODES)
firemode_colum = tf.feature_column.indicator_column(firemode_colum)

#create list with all feature columns
columns = [tf.feature_column.numeric_column(param) for param in NUMERICAL_PARAMS]
columns.append(type_column)
columns.append(firemode_colum)

#make sure you know whats going on inside
cols_to_vars_dict = {}

#create an input layer with your feature columns and make sure you get the cols_to_vars dictionary!
inputs = tf.feature_column.input_layer(features, columns, cols_to_vars=cols_to_vars_dict)

#shuffle the whole input layer
tf.set_random_seed(19071991)
inputs = tf.random_shuffle(inputs)

#print(cols_to_vars_dict)
#print(inputs)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer() #needed for the feature columns



In [42]:
genDict = {}

def addGeneratedWeapon(generatedWeaponTensorDict, dictToAdd):
    for key, value in generatedWeaponTensorDict.items():
        if key not in dictToAdd:
            dictToAdd[key] = []
        dictToAdd[key].append(value)
        
with tf.Session() as sess:
    sess.run((var_init, table_init))
    
    processedInput = sess.run(inputs)
    print("input size = %i" %len(processedInput))
    print(processedInput)
    tensorDict = tensorToParameterDict(processedInput[len(processedInput)-1], cols_to_vars_dict)
    addGeneratedWeapon(tensorDict, genDict)
    for key, value in tensorDict.items():
        print(key + " = " + value)
    
    
print(genDict) 
    

input size = 148
[[26.5 15.  11.  ...  0.   0.   0. ]
 [28.  21.  11.  ...  0.   0.   0. ]
 [80.  80.   0.  ...  1.   0.   0. ]
 ...
 [80.  80.   0.  ...  1.   0.   0. ]
 [53.  15.   6.  ...  0.   0.   0. ]
 [53.  21.  14.  ...  0.   0.   0. ]]
damages_first = 53.0
damages_last = 21.0
dmg_distances_first = 14.0
dmg_distances_last = 38.11
drag = 0.0025
firemode_Automatic = 0.0
firemode_Semi-Automatic = 1.0
hiprecoildec = 5.0
hiprecoilright = 1.0
hiprecoilup = 6.0
hordispersion = 0.0
initialspeed = 230.0
magsize = 6.0
reloadempty = 2.6
rof = 179.0
shotspershell = 1.0
type_Shotgun = 0.0
type_Pistol = 1.0
type_Rifle = 0.0
type_Submachine Gun = 0.0
type_Sniper Rifle = 0.0
verdispersion = 0.0
damages_first = 80.0
damages_last = 80.0
dmg_distances_first = 0.0
dmg_distances_last = 115.0
drag = 0.0025
firemode_Automatic = 0.0
firemode_Semi-Automatic = 0.0
hiprecoildec = 4.5
hiprecoilright = 1.0
hiprecoilup = 5.0
hordispersion = 0.0
initialspeed = 740.0
magsize = 10.0
reloadempty = 5.3999
rof = 